<a href="https://colab.research.google.com/github/Swapn2/PyTorch/blob/main/hyperparameter_tunning_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [23]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [24]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [25]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
X_train = X_train/255.0
X_test = X_test/255.0

In [29]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [30]:
train_dataset = CustomDataset(X_train, y_train)

In [31]:
test_dataset = CustomDataset(X_test, y_test)

1396

In [32]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate):

    super().__init__()

    layers = []

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(input_dim, neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer, output_dim))

    self.model = nn.Sequential(*layers)

  def forward(self, x):

    return self.model(x)

In [33]:
!pip install optuna

In [34]:
# objective function
def objective(trial):

  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 784
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimizer selection
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

  if optimizer_name == 'Adam':
    optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  # training loop

  for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()


  # evaluation
  model.eval()
  # evaluation on test data
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy

In [35]:
import optuna

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

[I 2025-07-19 06:36:41,366] A new study created in memory with name: no-name-7c0f2a0b-c0c4-4989-8622-d64ad2b29e2e
[I 2025-07-19 06:37:39,188] Trial 0 finished with value: 0.88325 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 112, 'epochs': 10, 'learning_rate': 0.0002681236002945675, 'dropout_rate': 0.30000000000000004, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 0.0004811237326969743}. Best is trial 0 with value: 0.88325.
[I 2025-07-19 06:38:03,462] Trial 1 finished with value: 0.8205 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 8, 'epochs': 10, 'learning_rate': 0.005294462485757652, 'dropout_rate': 0.30000000000000004, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 2.346647639705376e-05}. Best is trial 0 with value: 0.88325.
[I 2025-07-19 06:39:51,713] Trial 2 finished with value: 0.87825 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 104, 'epochs': 50, 'learning_rate': 0.0003552921677464318, 'dropout_rate': 0.1,

In [38]:
study.best_value


0.88325

In [37]:
study.best_params

{'num_hidden_layers': 3,
 'neurons_per_layer': 112,
 'epochs': 10,
 'learning_rate': 0.0002681236002945675,
 'dropout_rate': 0.30000000000000004,
 'batch_size': 32,
 'optimizer': 'RMSprop',
 'weight_decay': 0.0004811237326969743}